# **Ejercicios de pair programming Módulo 2 Sprint 1**
## **Estadística: Lección 3**
### **Tablas de contingencia, correlación, sesgos e intervalos de confianza**

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import skew
from pydataset import data

In [ ]:
games = pd.read_csv("data/games.csv")

1. Extraer los siguentes estadísiticos del conjunto de datos para varias de las varibles disponibles:
- Tabla de contingencia


- Coeficiente de correlación de las variables numéricas del dataset.


- Sesgos de alguna de las variables numéricas del dataset.


- Intervalo de confianza de algunas de las variables numéricas del dataset


2. Interpretación de los resultados.
- Ahora interpreta los resultados obtenidos de los diferentes estadísticos estudiados haciendo hincapié en aspectos interesantes de tus datos y explica lo mejor posible la naturaleza de los mismos.